# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [6]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [7]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [126]:
group = data.groupby(["CustomerID","ProductName"]).sum("Quantity")

In [127]:
group.head()

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [102]:
pivot = pd.pivot_table(group,"Quantity","ProductName","CustomerID").fillna(0)

In [103]:
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [104]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)
display(distances)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [106]:
similarities = distances[33]

In [114]:
top5 = similarities.sort_values(ascending=False)[1:6].index
print(top5)

Int64Index([264, 3535, 3317, 2503, 3305], dtype='int64', name='CustomerID')


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [136]:
group1 = group.reset_index()

In [150]:
group2 = group1.loc[group1["CustomerID"].isin(top5)]
group2

,CustomerID,ProductName,SalesID,ProductID,Quantity
131,264,Apricots - Halves,3730301,278,1
132,264,Apricots Fresh,5035449,158,1
133,264,Bacardi Breezer - Tropical,756802,49,1
134,264,Bagel - Plain,6728381,262,1
135,264,Banana - Leaves,6298459,271,1
...,...,...,...,...,...
3003,3535,Tray - 16in Rnd Blk,6122047,166,1
3004,3535,Wanton Wrap,2643670,253,1
3005,3535,Wine - Blue Nun Qualitatswein,3314104,269,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",6563480,192,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [161]:
rank = group2.groupby("ProductName").agg({"Quantity":sum})



In [165]:
ranked = rank.sort_values(by= "Quantity",ascending=False)
display(ranked)

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [172]:
pivot1 = pivot.reset_index()

In [191]:
ranked1 = ranked.reset_index()

In [197]:
merge = pd.merge(ranked1, pivot1, on='ProductName')
display(merge)

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,50.0,0.0,25.0,0.0,0.0,0.0,0.0
2,Towels - Paper / Kraft,3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Soup - Campbells Bean Medley,3,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Hinge W Undercut,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0
214,Ice Cream Bar - Hageen Daz To,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0
215,Jagermeister,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
216,Jolt Cola - Electric Blue,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [198]:
not_purchased =  merge[33]==0.0
merged_not_pur = merge[not_purchased]

In [200]:
merged_not_pur.head()

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,50.0,0.0,25.0,0.0,0.0,0.0,0.0
3,Soup - Campbells Bean Medley,3,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
6,Chicken - Soup Base,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [210]:
recommendations = {}
cust_id = sorted(list(data['CustomerID'].unique()))
for x in cust_id:
    similarities = distances[x].sort_values(ascending=False)[1:6]
    array = []
    for x in similarities.reset_index()['CustomerID']:
        array.append(x)
    sim_cust = group2.loc[group2['CustomerID'].isin(array)]
    array = []
    merge = pd.merge(ranked, pivot, on='ProductName')
    not_purchased =  merge[x]==0.0
    merged_not_pur = merge[not_purchased]
    final = merged_not_pur['Quantity'][:5].reset_index()
    list_of_names = final['ProductName'].to_list()
    part_dic ={x:list(final.ProductName)}
    recommendations.update(part_dic)

    
    
    
    
  


    
    
    




In [211]:

recommendations

{3305: ['Butter - Unsalted',
  'Soup - Campbells Bean Medley',
  'Pepper - Black, Whole',
  'Chocolate - Dark',
  'Raspberries - Fresh'],
 3909: ['Butter - Unsalted',
  'Wine - Ej Gallo Sierra Valley',
  'Towels - Paper / Kraft',
  'Wine - Blue Nun Qualitatswein',
  'Pepper - Black, Whole'],
 3535: ['Butter - Unsalted',
  'Wine - Ej Gallo Sierra Valley',
  'Chocolate - Dark',
  'Raspberries - Fresh',
  'Coffee Decaf Colombian'],
 2617: ['Butter - Unsalted',
  'Wine - Ej Gallo Sierra Valley',
  'Towels - Paper / Kraft',
  'Soup - Campbells Bean Medley',
  'Wine - Blue Nun Qualitatswein'],
 639: ['Butter - Unsalted',
  'Wine - Ej Gallo Sierra Valley',
  'Towels - Paper / Kraft',
  'Soup - Campbells Bean Medley',
  'Wine - Blue Nun Qualitatswein'],
 649: ['Butter - Unsalted',
  'Wine - Ej Gallo Sierra Valley',
  'Towels - Paper / Kraft',
  'Soup - Campbells Bean Medley',
  'Wine - Blue Nun Qualitatswein'],
 3317: ['Towels - Paper / Kraft',
  'Wine - Blue Nun Qualitatswein',
  'Chicken - S

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [214]:
df2 = pd.DataFrame.from_dict(recommendations,orient='index',columns=['Prd_rec1','Prd_rec2','Prd_rec3','Prd_rec4','Prd_rec5'])
df2.head()

,Prd_rec1,Prd_rec2,Prd_rec3,Prd_rec4,Prd_rec5
3305,Butter - Unsalted,Soup - Campbells Bean Medley,"Pepper - Black, Whole",Chocolate - Dark,Raspberries - Fresh
3909,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole"
3535,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Chocolate - Dark,Raspberries - Fresh,Coffee Decaf Colombian
2617,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein
639,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [218]:
distances1 = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'cosine'))), 
                         index=pivot.columns, columns=pivot.columns)
display(distances1)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [220]:
recommendations1= {}
cust_id = sorted(list(data['CustomerID'].unique()))
for x in cust_id:
    similarities = distances1[x].sort_values(ascending=False)[1:6]
    array = []
    for x in similarities.reset_index()['CustomerID']:
        array.append(x)
    sim_cust = group2.loc[group2['CustomerID'].isin(array)]
    array = []
    merge = pd.merge(ranked, pivot, on='ProductName')
    not_purchased =  merge[x]==0.0
    merged_not_pur = merge[not_purchased]
    final = merged_not_pur['Quantity'][:5].reset_index()
    list_of_names = final['ProductName'].to_list()
    part_dic ={x:list(final.ProductName)}
    recommendations1.update(part_dic)


In [222]:
df3 = pd.DataFrame.from_dict(recommendations1,orient='index',columns=['Prd_rec1','Prd_rec2','Prd_rec3','Prd_rec4','Prd_rec5'])
df3.head()

,Prd_rec1,Prd_rec2,Prd_rec3,Prd_rec4,Prd_rec5
60862,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Cod - Black Whole Fillet
98069,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein
2617,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein
92139,Butter - Unsalted,Towels - Paper / Kraft,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole"
25013,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Towels - Paper / Kraft,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein
